# Covid-19 Case Surveillance Public Use Dataset

Discover Demographic Trends of COVID-19 cases in the US reported to CDC



## Introduction

As you all know the problem created by Covid-19 pandemic and the life it took. Here we are doing some analysis on the covid 19 public dataset shared in Kaggle which gave information about the covid cases from US on 2020 and trying to get some insights from this data

Data set used here : [Kaggle](https://www.kaggle.com/arashnic/covid19-case-surveillance-public-use-dataset/download)

We are analysying the following Questions

     1. Find the case distribution across genders , age group and Demographic
     2. Death rate is higher is which age group?
     3. Find the trend of covid across time
     
Finally we are trying to use ARIMA time series prrediction model and predicting covid deaths for next month


In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

import sklearn
from sklearn.linear_model import LinearRegression

import scipy
from scipy import stats

import statsmodels
import statsmodels.api as sm

import watermark

#### Reading data

Enabling low_memory=False to deal with large datasets

In [40]:
df = pd.read_csv('COVID-19_Case_Surveillance_Public_Use_Data.csv',low_memory=False)


### 1. Business understanding 

Here we are analysing covid data set and trying to get some insights which helpful to reduce death . As a start we are trying to get answers to the following questions. 

 1. Find the case distribution across genders , age group and Demographic
 2. Death rate is higher is which age group?
 3. Find the trend of covid across time


### 2. Data Understanding 

In [41]:
df.head()

,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,medcond_yn
0,2020/11/10,2020/11/10,NaN,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,Unknown,No,No
1,2020/11/14,2020/11/10,2020/11/10,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No
2,2020/11/19,2020/11/10,2020/11/09,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No
3,2020/11/14,2020/11/10,NaN,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",Missing,Missing,No,Missing
4,2020/11/13,2020/11/10,2020/11/10,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,Yes


[Kaggle data set reference](https://www.kaggle.com/arashnic/covid19-case-surveillance-public-use-dataset)

#### Column Discription

| column                     | Description |
| -----------                | ----------- |
| cdc_report_dt              | Date cdc reported       |
| pos_spec_dt                | Date of first positive specimen collection (MM/DD/YYYY)|
| onset_dt              | What was the onset date?       |
| current_status              | What is the current status of this person?      |
| sex              | Gender      |
| age_group			| Age group categories       |
| Race and ethnicity (combined)              | Case Demographic     |
| hosp_yn              | Was the patient hospitalized?      |
| icu_yn              | Was the patient admitted to an intensive care unit (ICU)?    |
| death_yn              | Did the patient die as a result of this illness?      |
| medcond_yn              | Date cdc reported       |

In [42]:
df.describe() 

,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,medcond_yn
count,8405079,2870789,4395957,8405079,8405061,8404990,8405072,8405079,8405079,8405079,8405079
unique,321,313,338,2,5,10,9,4,4,4,4
top,2020/06/24,2020/11/09,2020/11/02,Laboratory-confirmed case,Female,20 - 29 Years,Unknown,No,Missing,No,Missing
freq,155857,50595,49527,7910118,4340898,1635264,2781176,3502556,6178281,3690813,6063390


No records in data

In [43]:
num_rows = df.shape[0] 
num_cols = df.shape[1]
num_rows,num_cols

(8405079, 11)

Column having no missing vallues

In [44]:
no_nulls = set(df.columns[df.isnull().mean()==0])
no_nulls

{'cdc_report_dt',
 'current_status',
 'death_yn',
 'hosp_yn',
 'icu_yn',
 'medcond_yn'}

Columns having missing values in other columns

# 3. Prepare Data

## Qn 1. Find the case distribution across genders , age group and Demographic

Covid Case status in data

In [45]:
count_vals = df.current_status.value_counts()
count_vals

Laboratory-confirmed case    7910118
Probable Case                 494961
Name: current_status, dtype: int64

Cases count by age group

In [46]:
df.age_group.value_counts()

20 - 29 Years    1635264
30 - 39 Years    1372623
40 - 49 Years    1267350
50 - 59 Years    1227493
60 - 69 Years     863260
10 - 19 Years     841450
70 - 79 Years     483689
80+ Years         382869
0 - 9 Years       299040
Unknown            31952
Name: age_group, dtype: int64

renaming the column "Race and ethnicity (combined)" to  'demographic for better usage

In [47]:
df = df.rename(columns={'Race and ethnicity (combined)': 'demographic'})


In [48]:
df.columns

Index(['cdc_report_dt', 'pos_spec_dt', 'onset_dt', 'current_status', 'sex',
       'age_group', 'demographic', 'hosp_yn', 'icu_yn', 'death_yn',
       'medcond_yn'],
      dtype='object')

Cases distribution by gender

In [49]:
df.sex.value_counts()

Female     4340898
Male       3970591
Unknown      73845
Missing      19520
Other          207
Name: sex, dtype: int64

In [50]:
values = df['current_status'].value_counts().tolist()

Graphical Representation of case status

In [51]:
values = df['current_status'].value_counts().tolist()
names = ["Confirmed", "Prabable"]

fig = px.pie(
    names=names,
    values=values,
    title="Case Status Distribution",
    color_discrete_sequence=px.colors.sequential.RdBu,
)
fig.show()

Graphical representation of cases distributions by gender

In [52]:
values = df['sex'].value_counts().tolist()
names = ["Female","Male", "Unknown","Missing","Other"]

  
fig = px.pie(
    names=names,
    values=values,
    title="Covid Case Gender Distribution",
    color_discrete_sequence=px.colors.sequential.RdBu,
)
fig.show()

In [53]:
values = df['age_group'].value_counts().tolist()
names = ["20 - 29 Years","30 - 39 Years","40 - 49 Years","50 - 59 Years","60 - 69 Years","10 - 19 Years","70 - 79 Years","80+ Years","0 - 9 Years","Unknown",]
fig = px.pie(
    names=names,
    values=values,
    title="Case Status Distribution",
    color_discrete_sequence=px.colors.sequential.RdBu,
)
fig.show()


Graphiccal representation by age group

In [54]:
fig = px.bar(
    x=names,
    y=values,
    title="Age Group Distribution",
    labels={
        'x': 'Age Group',
        'y': 'Number of Patients'
    },
    color=values
)
fig.show()

Graphical representation of demographics

In [55]:
values = df['demographic'].value_counts()#.tolist()
names = list(values.index) 

fig = px.bar(
    x=names,
    y=values,
    title="Demographic Distribution",
    labels={
        'x': 'Demographic',
        'y': 'Number of Patients'
    },
    color=values
)
fig.show()

## Qn 2 . Death rate is higher is which age group?

Finding which age group have higeset no of death reported as well the death rate is very higher

In [56]:
df_death = df[['age_group','death_yn','current_status']].copy()
df_death['is_dead'] = df_death['death_yn'].map(lambda x: 1 if x == "Yes" else 0)
df_death['is_confirmed'] = df_death['current_status'].map(lambda x: 1 if x == "Laboratory-confirmed case" else 0)

df_death_grouped = df_death.groupby(['age_group']).agg({'is_dead':'sum','is_confirmed':'sum'})
df_death_grouped = df_death_grouped.reset_index()

In [57]:
df_death_grouped["death_rate"] =  df_death_grouped["is_dead"] / df_death_grouped["is_confirmed"] *100
df_death_grouped

,age_group,is_dead,is_confirmed,death_rate
0,0 - 9 Years,71,277058,0.025626
1,10 - 19 Years,135,780198,0.017303
2,20 - 29 Years,850,1547427,0.054930
3,30 - 39 Years,2261,1300297,0.173883
4,40 - 49 Years,5397,1195038,0.451617
5,50 - 59 Years,13973,1154470,1.210339
6,60 - 69 Years,29957,813252,3.683606
7,70 - 79 Years,45450,458427,9.914338
8,80+ Years,85461,363901,23.484684
9,Unknown,15,19969,0.075116


In [58]:
values = df_death_grouped['is_dead'].tolist()
names = df_death_grouped['age_group'].tolist() 
fig = px.pie(
    names=names,
    values=values,
    title="Death Distribution",
    color_discrete_sequence=px.colors.sequential.RdBu,
)
fig.show()

## Qn 3 . Find the trend of covid across time

Cleaning the date field

In [59]:
df['cdc_report_dt'] = pd.to_datetime(df.cdc_report_dt)


In [60]:
df_time_series = df[['cdc_report_dt', 'current_status','death_yn']].copy()


Creating new columns for cleaned values for death,confirmeed cases and probable cases

In [61]:
df_time_series['is_dead'] = df_time_series['death_yn'].map(lambda x: 1 if x == "Yes" else 0)
df_time_series['is_confirmed'] = df_time_series['current_status'].map(lambda x: 1 if x == "Laboratory-confirmed case" else 0)
df_time_series['is_probable'] = df_time_series['current_status'].map(lambda x: 1 if x == "Probable Case" else 0)


Groupingg by date

In [62]:
df_time_series_grouped = df_time_series.groupby(['cdc_report_dt']).agg({'is_dead':'sum','is_confirmed':'sum','is_probable':'sum'})
df_time_series_grouped = df_time_series_grouped.reset_index()

In [63]:
fig = px.line(df_time_series_grouped, x='cdc_report_dt', y=["is_confirmed","is_dead","is_probable"])
fig.show()

By looking the graph we can see that covid cases is exponetiallly increasing across the given date frame also death we can see almost like constant in days